<h1>Proyecto 7 - Automatización de sistemas de distribución<h1>

<h2>Autor: Juan Sebastián Caicedo & César Cabrales<h2>

In [15]:
import numpy as np
from numpy.linalg import inv
from scipy.optimize import minimize
from tabulate import tabulate

In [31]:
sc_mva = 500  # MVA de cortocircuito
alpha = 3
beta = 3
kv_ll = 12.47  # kV

# Cálculo de impedancia de secuencia
z1 = kv_ll**2 / sc_mva
r1 = z1 / np.sqrt(1 + alpha**2)
x1 = r1 * alpha
z1_cplx = complex(r1, x1)
i_sc = kv_ll**2 / (z1 * np.sqrt(3))

# Más cálculos de impedancia
gamma = (3 * (kv_ll**2)) / sc_mva
coef_a = beta**2 + 1
coef_b = 4 * r1 + 4 * x1 * beta
coef_c = 4 * (r1**2) + 4 * (x1**2) - gamma**2
roots = np.roots([coef_a, coef_b, coef_c])
r0 = roots[1]
x0 = r0 * beta
z0_cplx = complex(r0, x0)
z2_cplx = z1_cplx

# Matriz Z de impedancias de secuencia
z012 = np.diag([z0_cplx, z1_cplx, z2_cplx])
d_factor = -0.5 + (np.sqrt(3)/2) * 1j
d_mtx = np.array([[1, 1, 1], [1, d_factor**2, d_factor], [1, d_factor, d_factor**2]])
z_base = (12.47**2) / 6  # Base de impedancia en ohmios
z_g = (d_mtx @ z012 @ inv(d_mtx)) / z_base

In [32]:
# Matrices de línea
dist12 = 2000 * 0.000189  # MILLAS
dist34 = 2500 * 0.000189  # MILLAS
zt_unit = 0.01 + 1j * 0.06
z_trans = np.diag([zt_unit, zt_unit, zt_unit])

imp_base_l = (4.16**2) / 6
z12_imp = np.array([
    [0.4576 + 1j*1.0780, 0.1560 + 1j*0.5017, 0.1535 + 1j*0.3849],
    [0.1560 + 1j*0.5017, 0.4666 + 1j*1.0482, 0.1580 + 1j*0.4236],
    [0.1535 + 1j*0.3849, 0.1580 + 1j*0.4236, 0.4615 + 1j*1.0651]
])
z12_total = z12_imp * dist12 / z_base
z34_total = z12_imp * dist34 / imp_base_l

# Y-bus
y_gen = inv(z_g)
y_12 = inv(z12_total)
y_34 = inv(z34_total)
y_trans = inv(z_trans)
y_35 = y_34
y_45 = y_34
zero_mtx = np.zeros((3, 3), dtype=complex)
y_bus = np.block([
    [y_gen, -y_gen, zero_mtx, zero_mtx, zero_mtx, zero_mtx],
    [-y_gen, y_gen + y_12, -y_12, zero_mtx, zero_mtx, zero_mtx],
    [zero_mtx, -y_12, y_12 + y_trans, -y_trans, zero_mtx, zero_mtx],
    [zero_mtx, zero_mtx, -y_trans, y_trans + y_35 + y_34, -y_34, -y_35],
    [zero_mtx, zero_mtx, zero_mtx, -y_34, y_34 + y_45, -y_45],
    [zero_mtx, zero_mtx, zero_mtx, -y_35, -y_45, y_35 + y_45]
])


In [33]:
# Demanda y condiciones de slack bus
pd = np.array([1275, 1800, 2375]) / 2000
sd = pd / np.array([0.85, 0.90, 0.95])
qd = np.sqrt(sd**2 - pd**2)
v1 = np.array([1, 1, 1])  # slack bus voltages
theta1 = np.array([0, -120, 120]) * np.pi / 180  # slack bus angles

# Definición de límites
lower_bounds = [0.5] * 15 + [-2 * np.pi] * 15 + [0] * 3 + [-10] * 6
upper_bounds = [1.5] * 15 + [2 * np.pi] * 15 + [5] * 3 + [10] * 6
bounds = list(zip(lower_bounds, upper_bounds))

x0 = np.concatenate((np.ones(15), np.tile(theta1, 5), np.zeros(3), np.zeros(6)))


In [34]:
def objective(x):
    # Usando variables globales para simplificar el ejemplo
    global pd
    Plossa = x[33] + (1000 / 2000) - pd[0]
    Plossb = x[34] + (1000 / 2000) - pd[1]
    Plossc = x[35] + (1000 / 2000) - pd[2]
    return Plossa + Plossb + Plossc

def objective2(x):
    global v1
    return abs(v1[0] - x[9]) + abs(v1[1] - x[10]) + abs(v1[2] - x[11])

def constraints(x):
    global pd, qd, ybus, v1, theta1
    V = np.zeros(18, dtype=complex)  # Total of 18 voltages: 3 slack and 15 others
    V[:3] = v1 * np.exp(1j * theta1)  # Setting slack bus voltages
    for i in range(15):
        V[i+3] = x[i] * np.exp(1j * x[i+15])  # Voltage magnitudes and angles for other buses

    I0 = np.array([np.conj(complex(x[33+j], x[36+j]) / V[j]) for j in range(3)])  # Currents for generators?
    I4 = -np.array([np.conj(complex(pd[j], qd[j]) / V[j+12]) for j in range(3)])  # Currents for loads
    I5 = np.array([np.conj(complex(1000/2000, x[30+j]) / V[j+15]) for j in range(3)])  # Additional currents

    I = np.concatenate([I0, np.zeros(3, dtype=complex), np.zeros(3, dtype=complex), np.zeros(3, dtype=complex), I4, I5])

    kvl = I - ybus @ V  # Kirchhoff Voltage Law

    # Concatenate real and imaginary parts of the KVL equation to form equality constraints
    Ceq = np.concatenate([kvl.real, kvl.imag])

    # Return Ceq such that all entries should be zero (hence, 'eq' type)
    return Ceq

# Update the minimize call with the proper constraint format
result = minimize(objective, x0, bounds=bounds, constraints={'type': 'eq', 'fun': constraints})

print("Función minimizada:", result.fun)  # Print the function value at the optimum


Función minimizada: 0.11299030674935573


Minimización de diferencia de voltaje

In [35]:
result2 = minimize(objective2, x0, bounds=bounds, constraints={'type': 'eq', 'fun': constraints})

In [40]:
x0 = result.x
x02 = result2.x

In [41]:
indicators = [
    "mV1a", "mV1b", "mV1c", "mV2a", "mV2b", "mV2c",
    "mV3a", "mV3b", "mV3c", "mV4a", "mV4b", "mV4c",
    "mV5a", "mV5b", "mV5c", "aV1a", "aV1b", "aV1c",
    "aV2a", "aV2b", "aV2c", "aV3a", "aV3b", "aV3c",
    "aV4a", "aV4b", "aV4c", "aV5a", "aV5b", "aV5c",
    "Qca", "Qcb", "Qcc", "P1a", "P1b", "P1c", "Q1a", "Q1b", "Q1c",
    "Plossina", "Plossinb", "Plossinc", "Total Losses"
]

# Crear valores para la tabla
values = [
    *x0[:15], *(x0[15:30] * 180 / np.pi), x0[30] * 2, x0[31] * 2, x0[33] * 2,
    x0[33] * 2, x0[34] * 2, x0[35] * 2, x0[36] * 2, x0[37] * 2, x0[38] * 2,
    (x0[33] * 2 + 1 - pd[0] * 2),
    (x0[34] * 2 + 1 - pd[1] * 2),
    (x0[35] * 2 + 1 - pd[2] * 2),
    result.fun * 2000
]

values2 = [
    *x02[:15], *(x02[15:30] * 180 / np.pi), x02[30] * 2, x02[31] * 2, x02[33] * 2,
    x02[33] * 2, x02[34] * 2, x02[35] * 2, x02[36] * 2, x02[37] * 2, x02[38] * 2,
    (x02[33] * 2 + 1 - pd[0] * 2),
    (x02[34] * 2 + 1 - pd[1] * 2),
    (x02[35] * 2 + 1 - pd[2] * 2),
    result2.fun * 2000
]

# Crear tabla
table = [[ind, val1, val2] for ind, val1, val2 in zip(indicators, values, values2)]
print(tabulate(table, headers=["Indicator", "Power Loss Minimization", "Voltage Deviation Minimization"], tablefmt="grid"))

+--------------+---------------------------+----------------------------------+
| Indicator    |   Power Loss Minimization |   Voltage Deviation Minimization |
+==============+===========================+==================================+
| mV1a         |                 0.997982  |                        1.00193   |
+--------------+---------------------------+----------------------------------+
| mV1b         |                 0.997988  |                        1.00428   |
+--------------+---------------------------+----------------------------------+
| mV1c         |                 0.995962  |                        1.00177   |
+--------------+---------------------------+----------------------------------+
| mV2a         |                 0.997262  |                        1.00299   |
+--------------+---------------------------+----------------------------------+
| mV2b         |                 0.993375  |                        1.0049    |
+--------------+------------------------